# Fantasy Football Prep

The League I'm in has the following parameters:
1. 1 QB
2. 2 RB
3. 3 WR
4. 1 TE 
5. 1 flex
6. 1 D/ST
7. 1 K 

In [ ]:
def check_rules(roster,rules):
    
    # do a uniqueness check?

In [1]:
# boilerplate imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot





In [2]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator




In [24]:
def calculate_snake(pos,rounds=12):
    double = [pos,13.-pos]
    order = []
    for rnd in range(0,rounds,2):
        order.extend(double)
        
    return 12.*(np.arange(0.,float(rounds),1.)) + np.array(order)




In [69]:
r  = requests.get('https://www.fantasypros.com/nfl/rankings/half-point-ppr-cheatsheets.php')

# standard league
#https://www.fantasypros.com/nfl/rankings/consensus-cheatsheets.php

# [u'Rank\r\n\r\n', u'Player (Team)', u'Pos', u'Bye', \
#         0              1              2       3
#     u'Best', u'Worst', u'Avg', u'Std Dev', u'ADP', u'vs. ADP', u'Notes']
#        4          5      6        7           8         9        10


data = r.text

soup = BeautifulSoup(data)

tables = soup.findAll('table')

ordering = calculate_snake(5.)

ordinal = 6

listing = []
player = []
position = []
notes = []
for indx,table in enumerate(tables): 
    
    #print indx,[th.get_text() for th in table.find("tr").find_all("th")]
   
    if indx==0:
        
        #print [th.get_text() for th in table.find("tr").find_all("th")]
        
        for row in table.find_all("tr")[1:]:
            sav = [td.get_text() for td in row.find_all("td")]
            # get the links here and then think about scraping those pages
            
            #try:
            #    sav2 = [td.find('a') for td in row.find_all("td")][1]
            #    print 'https://www.fantasypros.com'+sav2.get('href')
            #except:
            #    pass
            
            try:
                #if float(sav[0]) in ordering:
                #    print sav[0],sav[1],sav[2]#,sav[10]
                listing.append(float(sav[ordinal]))
                player.append(sav[1])
                position.append(sav[2])
                notes.append(sav[10])
                
            except:
                pass#print 'fail'
                
            
NU = np.array(listing)
PL = np.array(player)
PO = np.array(position)
NO = np.array(notes)
# organize as specified
reorg = NU.argsort()
                    
for entry,value in enumerate(NU[reorg]):
    
    if (float(entry) + 1.) in ordering:
        print NU[reorg[entry]],PL[reorg[entry]],PO[reorg[entry]]
        print ''
        print NO[reorg[entry]]
        print ''
        


#https://www.fantasypros.com/nfl/players/david-johnson-rb.php

# replace 'players' with 'stats'

5.4 Julio Jones ATL  WR3

Julio is #1 in NFL history with 96.3 yards per game. With that being said, he is an increased injury risk over Brown and Beckham, plus he has been known to dis

20.3 Doug Baldwin SEA  WR11

He's now finished as a top-10 wide receiver in back-to-back seasons, and that was despite Russell Wilson taking a slight step back last year, though it was likely injury-related. His target numbers have gone up four straight seasons, as have the Seahawks pass attempts. They are attempting to get their run-game going again, but Baldwin is who moves the sticks.

34.6 Aaron Rodgers GB  QB1

He's a machine, plain and simple. Year-over-year Rodgers builds his hall of fame resume, and it translates to fantasy, considering he's finished as the No. 1 or No. 2 quarterback in seven of nine seasons (was also No. 1 before getting hurt in 2013) since taking over for Brett Favre. There's so much uncertainty around this part of the rankings, he's simply a safer pick.

45.4 Golden Tate DET

In [67]:
r  = requests.get('https://www.fantasypros.com/nfl/players/tom-brady.php')

# standard league
#https://www.fantasypros.com/nfl/rankings/consensus-cheatsheets.php

# [u'Rank\r\n\r\n', u'Player (Team)', u'Pos', u'Bye', \
#         0              1              2       3
#     u'Best', u'Worst', u'Avg', u'Std Dev', u'ADP', u'vs. ADP', u'Notes']
#        4          5      6        7           8         9        10


data = r.text

soup = BeautifulSoup(data)

tables = soup.findAll('table')

for indx,table in enumerate(tables): 
    
    print indx,[th.get_text() for th in table.find("tr").find_all("th")]
    
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Rushing Stats']:
        
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Receiving Stats']:
        
    #if [th.get_text() for th in table.find("tr").find_all("th")]==[u'Passing Stats']:



        #print [th.get_text() for th in table.find("tr").find_all("th")]
        
    for row in table.find_all("tr")[1:]:
            sav = [td.get_text() for td in row.find_all("td")]
            
            print sav
   

# receiving
# SEASON	TEAM	GAMES	REC	TGT	YDS	Y/R	LG	TD

# rushing 
# SEASON	TEAM	GAMES	ATT	YDS	Y/A	LG	TD	FUM	FUML

0 []
1 [u' ', u'ECR', u'Best', u'Worst', u'# of Experts']
[u'Draft', u'QB #2', u'QB #1', u'QB #8', u'104']
[u'Dynasty ', u'QB #10', u'QB #6', u'QB #18', u'21']
2 [u' ', u'Consensus', u'CBS', u'Fantrax', u'NFL', u'DW', u'FFC']
[u'Overall', u'#17', u'#14', u'#23', u'#23', u'#24', u'#30']
[u'Quarterbacks', u'#2', u'#2', u'#2', u'#2', u'#2', u'#2']
3 [u'Pass Cmp', u'Pass Att', u'Pass Yds', u'Pass Tds', u'Pass Ints', u'Rush Att', u'Rush Yds', u'Rush Tds', u'Fumbles', u'Points']
[u'393.4', u'601.8', u'4697.4', u'35.4', u'8.3', u'35.5', u'85.3', u'1.3', u'2.8', u'323.9']
4 [u' ', u'Pass Yds', u'Pass Tds', u'Rush Yds', u'Rush Tds', u'Points']
[u'Last Week', u'276', u'3', u'6', u'0', u'25.6']
[u'Last 3 Weeks', u'226', u'2', u'1.7', u'0', u'17.9']
[u'Season to Date', u'296.2', u'2.3', u'5.3', u'0', u'21.5']
5 [u' ', u'OPP', u'Pass Yds', u'Pass Tds', u'Rush Yds', u'Rush Tds', u'Points']
[u'Week 17', u'@ MIA', u'276', u'3', u'6', u'0', u'25.6']
[u'Week 16', u'vs. NYJ', u'214', u'3', u'0', u'0', u'